In [23]:
%use lets-plot

In [24]:
import kotlinx.serialization.json.Json
import kotlinx.serialization.json.JsonNamingStrategy
import io.github.andreypfau.tondhtcrawler.*
import java.io.File

@Suppress("OPT_IN_USAGE")
val TL_JSON = Json {
    ignoreUnknownKeys = true
    classDiscriminator = "@type"
    useArrayPolymorphism = false
    namingStrategy = JsonNamingStrategy.SnakeCase
    prettyPrint = true
}

val crawlResults = TL_JSON.decodeFromString<List<DhtCrawlResult>>( File("crawl-results.json").readText())

In [25]:
import kotlinx.io.bytestring.ByteString

fun ByteString.toBitString(): String = this.toByteArray().joinToString("") { b ->
    (b.toInt() and 0xFF).toString(2).padStart(8, '0')
}
val a = 12

In [26]:
val depth = 8
val bitStrings = crawlResults.map { result ->
    result.info.id.idShort.hash.toBitString().take(depth)
}

In [27]:
val counts: Map<String, Int> = bitStrings
    .groupingBy { it }
    .eachCount()

In [28]:
val allPrefixes = (0 until (1 shl depth))
    .map { it.toString(2).padStart(depth, '0') }
    .sorted()

In [29]:
val regions     = allPrefixes
val populations = allPrefixes.map { counts[it] ?: 0 }

In [30]:
val expectedDensity = populations.average()
val riskThreshold   = expectedDensity + 3 * sqrt(expectedDensity)

In [31]:
val data = mapOf(
    "region"     to regions,
    "population" to populations
)

In [32]:
val plot = letsPlot(data) {
    x = "region"
    y = "population"
} +
        geomBar(
            stat = Stat.identity,    // <<< вот тут передаём Stat.identity
            fill = "#00cc99"
        ) +
        geomHLine(
            yintercept = expectedDensity,
            linetype    = "dashed",
            color       = "red"
        ) +
        geomHLine(
            yintercept = riskThreshold,
            linetype    = "dashed",
            color       = "orange"
        ) +
        ggtitle("Keyspace Regions Population (Depth $depth)") +
        xlab("Keyspace regions") +
        ylab("Population")
plot

00000000 
 
 
 
 
 
 
 
 
 00000111 
 
 
 
 
 
 
 
 
 00001110 
 
 
 
 
 
 
 
 
 00010101 
 
 
 
 
 
 
 
 
 00011100 
 
 
 
 
 
 
 
 
 00100011 
 
 
 
 
 
 
 
 
 00101010 
 
 
 
 
 
 
 
 
 00110001 
 
 
 
 
 
 
 
 
 00111000 
 
 
 
 
 
 
 
 
 00111111 
 
 
 
 
 
 
 
 
 01000110 
 
 
 
 
 
 
 
 
 01001101 
 
 
 
 
 
 
 
 
 01010100 
 
 
 
 
 
 
 
 
 01011011 
 
 
 
 
 
 
 
 
 01100010 
 
 
 
 
 
 
 
 
 01101001 
 
 
 
 
 
 
 
 
 01110000 
 
 
 
 
 
 
 
 
 01110111 
 
 
 
 
 
 
 
 
 01111110 
 
 
 
 
 
 
 
 
 10000101 
 
 
 
 
 
 
 
 
 10001100 
 
 
 
 
 
 
 
 
 10010011 
 
 
 
 
 
 
 
 
 10011010 
 
 
 
 
 
 
 
 
 10100001 
 
 
 
 
 
 
 
 
 10101000 
 
 
 
 
 
 
 
 
 10101111 
 
 
 
 
 
 
 
 
 10110110 
 
 
 
 
 
 
 
 
 10111101 
 
 
 
 
 
 
 
 
 11000100 
 
 
 
 
 
 
 
 
 11001011 
 
 
 
 
 
 
 
 
 11010010 
 
 
 
 
 
 
 
 
 11011001 
 
 
 
 
 
 
 
 
 11100000 
 
 
 
 
 
 
 
 
 11100111 
 
 
 
 
 
 
 
 
 11101110 
 
 
 
 
 
 
 
 
 11110101 
 
 
 
 
 
 
 
 
 11111100 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 8 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 12 
 
 
 
 
 
 
 
 
 Keyspace Regions Population (Depth 8) 
 
 
 
 
 Population 
 
 
 
 
 Keyspace regions

In [33]:
val address2results = crawlResults.groupBy { it.info.addrList.first().toString() }
val counts = address2results.mapValues { it.value.size }
val group = counts.entries.groupBy { it.value }
group.forEach {
    println("${it.key}: ${it.value.size}")
}

1: 1426
